In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

from source_code.SqlCo import Sqldd
from source_code.backpip import data_up_pip

Autosave disabled
Local Sec Env loaded


In [2]:
## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today() - timedelta(days=1)}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today() - timedelta(days=1)}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
319,AFG,Asia,Afghanistan,2021-01-08,53332.0,125.0,258.000,2257.0,4.0,9.429,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.498
625,ALB,Europe,Albania,2021-01-08,62378.0,673.0,580.286,1230.0,7.0,7.000,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.785
944,DZA,Africa,Algeria,2021-01-08,101657.0,275.0,251.429,2798.0,6.0,5.143,...,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88,0.754


In [3]:
## dump the file for using it in other nb
# c.to_csv(f"data\\Base_Files\\full_data{date.today()}.csv", index=False)

In [4]:
## insert yesterday reporting to bdd
# try:
#     verif_data = verif_data.replace({np.nan: 'Null'})
#     data_job = data_up_pip(verif_data) 
#     data_job.up_baseline()
#     data_job.clean_leave()
# except:
#     print("BDD job Failed -- Normal if you not have acces to main code")

In [5]:
## Load data from days -1 to evaluate the performances of model
yesterday_date = date.today() - timedelta(days=2)
yesterday_date = str(yesterday_date)
this_date = date.today() - timedelta(days=1)
this_date = str(this_date)

tip = Sqldd()
cnx, cursor = tip.get_bdd_co()
data_load = pd.read_sql(f"SELECT * FROM cov_aipred WHERE date='{yesterday_date}';", con=cnx) 

In [6]:
def Eval(c,data_load,country):
    """ function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    df = df[df['date'].isin([f"{this_date}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]

    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    res_tempo= res_tempo.reset_index()
    rez1 = res_tempo.at[0,"total_cases_predict"]
    rez2 = res_tempo.at[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2
    
    ind = 0    
    follow_df = pd.DataFrame()
    follow_df.at[ind,"date"] = v0
    follow_df.at[ind,"country"] = country
    follow_df.at[ind,"total_cases_predict"] = rez1
    follow_df.at[ind,"total_cases_real"] = v1
    follow_df.at[ind,"total_deaths_predict"] = rez2
    follow_df.at[ind,"total_deaths_real"] = v2
    follow_df.at[ind,"error_abs_cases"] = errorCase1
    follow_df.at[ind,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [7]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
# follow_df4 = Eval(c,data_load,"Belgium")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [8]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2021-01-08,France,2792420.0,2804743.0,67024.0,67566.0,-12323.0,-542.0
0,2021-01-08,China,96525.0,96562.0,4791.0,4789.0,-37.0,2.0
0,2021-01-08,Italy,2243170.0,2237890.0,77807.0,77911.0,5280.0,-104.0
0,2021-01-08,United States,21787200.0,21862773.0,366052.0,368773.0,-75573.0,-2721.0
0,2021-01-08,World,88806600.0,88860500.0,1904040.0,1913419.0,-53900.0,-9379.0
0,2021-01-08,United Kingdom,2945980.0,2966244.0,79393.0,79965.0,-20264.0,-572.0
0,2021-01-08,Germany,1911520.0,1905638.0,40048.0,40022.0,5882.0,26.0
0,2021-01-08,Iran,1282990.0,1274514.0,56564.0,56018.0,8476.0,546.0
0,2021-01-08,Turkey,2328560.0,2307581.0,22502.0,22450.0,20979.0,52.0
0,2021-01-08,Brazil,7975510.0,8013708.0,200818.0,201460.0,-38198.0,-642.0


In [9]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
# rapport.to_csv(f"data\\Rapport\\rap{date.today()}.csv")

In [9]:
# insert yesterday reporting to bdd
try:
    data_job = data_up_pip(rapport) 
    data_job.up_rapp()
    data_job.clean_leave()
except:
    print("BDD job Failed -- Normal if you not have acces to main code")

 Insert Rapport Done !! 
